In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pickle

In [2]:
train_dtype = {
    'Store':'int16',
    'DayOfWeek':'int16',
    'Date':'str',
    'Sales':'float',
    'Customers':'float',
    'Open':'int16',
    'Promo':'int16',
    'StateHoliday':'str',
    'SchoolHoliday':'int16'
}
test_dtype = {
    'Id':'int32',
    'Store':'int16',
    'DayOfWeek':'int16',
    'Date':'str',
    'Promo':'int16',
    'StateHoliday':'str',
    'SchoolHoliday':'int16'
}
parse_dates = ['Date']
store_col = ['Store', 'StoreType', 'Assortment']

In [3]:
store = pd.read_csv('../data/rossman_data/store.csv',usecols=store_col)
train = pd.read_csv('../data/rossman_data/train.csv',dtype=train_dtype,parse_dates=parse_dates)
test = pd.read_csv('../data/rossman_data/test.csv',dtype=test_dtype,parse_dates=parse_dates)
test.Open =test.copy().Open.fillna(0.0).astype('int16')
test['Sales']=np.zeros(test.shape[0])

In [4]:
train_store_set = set(train.Store)
test_store_set = set(test.Store)
store_list = []
for s in train_store_set:
    if s in test_store_set:
        store_list.append(s)
train = train[train.Store.isin(store_list)]

In [5]:
train.shape

(773231, 9)

In [6]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263.0,555.0,1,1,0,1
2,3,5,2015-07-31,8314.0,821.0,1,1,0,1
6,7,5,2015-07-31,15344.0,1414.0,1,1,0,1
7,8,5,2015-07-31,8492.0,833.0,1,1,0,1
8,9,5,2015-07-31,8565.0,687.0,1,1,0,1


In [7]:
test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,Sales
0,1,1,4,2015-09-17,1,1,0,0,0.0
1,2,3,4,2015-09-17,1,1,0,0,0.0
2,3,7,4,2015-09-17,1,1,0,0,0.0
3,4,8,4,2015-09-17,1,1,0,0,0.0
4,5,9,4,2015-09-17,1,1,0,0,0.0


In [8]:
store.head()

,Store,StoreType,Assortment
0,1,c,a
1,2,a,a
2,3,a,a
3,4,c,c
4,5,a,a


In [9]:
train['dom'] = train.Date.dt.day
test['dom'] = test.Date.dt.day
train['doy'] = train.Date.dt.dayofyear
test['doy'] = test.Date.dt.dayofyear
train['quarter'] = train.Date.dt.quarter
test['quarter'] = test.Date.dt.quarter
train['month'] = train.Date.dt.month
test['month'] = test.Date.dt.month
train['week'] = train.Date.dt.week
test['week'] = test.Date.dt.week

In [10]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,dom,doy,quarter,month,week
0,1,5,2015-07-31,5263.0,555.0,1,1,0,1,31,212,3,7,31
2,3,5,2015-07-31,8314.0,821.0,1,1,0,1,31,212,3,7,31
6,7,5,2015-07-31,15344.0,1414.0,1,1,0,1,31,212,3,7,31
7,8,5,2015-07-31,8492.0,833.0,1,1,0,1,31,212,3,7,31
8,9,5,2015-07-31,8565.0,687.0,1,1,0,1,31,212,3,7,31


In [11]:
train.Date.max(),train.Date.min()

(Timestamp('2015-07-31 00:00:00'), Timestamp('2013-01-01 00:00:00'))

In [12]:
# to get the train and validation marker as Id column, which is good to concat with test set
special_date = train.Date.iloc[49060]
ID = np.zeros(train.shape[0])
ID[train.Date>special_date] = np.ones(len(ID[train.Date>special_date]))*-50000 # validation set
ID[train.Date<=special_date] = np.ones(len(ID[train.Date<=special_date]))*50000 # train set
train['Id'] = ID.astype('int32')

In [13]:
train.shape,test.shape

((773231, 15), (41088, 14))

In [24]:
full_data = pd.concat([train.drop('Customers',axis=1)[::-1],test[::-1]],axis=0)

In [25]:
full_data.head()

,Date,DayOfWeek,Id,Open,Promo,Sales,SchoolHoliday,StateHoliday,Store,dom,doy,month,quarter,week
1017208,2013-01-01,2,50000,0,0,0.0,1,a,1115,1,1,1,1,1
1017207,2013-01-01,2,50000,0,0,0.0,1,a,1114,1,1,1,1,1
1017206,2013-01-01,2,50000,0,0,0.0,1,a,1113,1,1,1,1,1
1017205,2013-01-01,2,50000,0,0,0.0,1,a,1112,1,1,1,1,1
1017204,2013-01-01,2,50000,0,0,0.0,1,a,1111,1,1,1,1,1


In [26]:
full_data = full_data.merge(store,how='left',on='Store')

In [27]:
full_data.head()

,Date,DayOfWeek,Id,Open,Promo,Sales,SchoolHoliday,StateHoliday,Store,dom,doy,month,quarter,week,StoreType,Assortment
0,2013-01-01,2,50000,0,0,0.0,1,a,1115,1,1,1,1,1,d,c
1,2013-01-01,2,50000,0,0,0.0,1,a,1114,1,1,1,1,1,a,c
2,2013-01-01,2,50000,0,0,0.0,1,a,1113,1,1,1,1,1,a,c
3,2013-01-01,2,50000,0,0,0.0,1,a,1112,1,1,1,1,1,c,c
4,2013-01-01,2,50000,0,0,0.0,1,a,1111,1,1,1,1,1,a,a


In [28]:
avg_sales = full_data.groupby(['Store'],as_index=False)['Sales'].mean().rename(columns={'Sales':'Avg_Sales'})

In [29]:
full_data = full_data.merge(avg_sales,how='left',on='Store')

In [30]:
lbl = LabelEncoder()
for col in ['DayOfWeek','SchoolHoliday','StateHoliday','Store','dom','doy',
           'month','quarter','week','StoreType','Assortment']:
    full_data[col] = lbl.fit_transform(full_data[col])

In [31]:
full_data.reset_index(drop=True,inplace=True)

In [32]:
full_data.head()

,Date,DayOfWeek,Id,Open,Promo,Sales,SchoolHoliday,StateHoliday,Store,dom,doy,month,quarter,week,StoreType,Assortment,Avg_Sales
0,2013-01-01,1,50000,0,0,0.0,1,1,855,0,0,0,0,0,3,2,4971.948485
1,2013-01-01,1,50000,0,0,0.0,1,1,854,0,0,0,0,0,0,2,16366.247475
2,2013-01-01,1,50000,0,0,0.0,1,1,853,0,0,0,0,0,0,2,5248.729293
3,2013-01-01,1,50000,0,0,0.0,1,1,852,0,0,0,0,0,2,2,8054.842424
4,2013-01-01,1,50000,0,0,0.0,1,1,851,0,0,0,0,0,0,0,4132.400000


In [33]:
full_data['Store_'] = full_data['Store'].copy()

In [34]:
full_data['Sales'] = np.log1p(full_data.Sales.values)
full_data['Avg_Sales'] = np.log1p(full_data.Avg_Sales.values)

In [38]:
feature_list = ['DayOfWeek','Id','Open','Promo',
               'Sales','SchoolHoliday','StateHoliday','Store','dom',
               'doy','month','quarter','week','StoreType','Assortment','Avg_Sales']

In [44]:
feature_dict = {i:None for i in feature_list}
feature_index = {feat:i for i,feat in enumerate(feature_list)}

In [45]:
for k in feature_dict.keys():
    feature_dict[k]=full_data[['Store_','Date',k]].set_index(['Store_','Date']).unstack(-1,fill_value=0)
    print(k,feature_dict[k].shape)

DayOfWeek (856, 990)
Id (856, 990)
Open (856, 990)
Promo (856, 990)
Sales (856, 990)
SchoolHoliday (856, 990)
StateHoliday (856, 990)
Store (856, 990)
dom (856, 990)
doy (856, 990)
month (856, 990)
quarter (856, 990)
week (856, 990)
StoreType (856, 990)
Assortment (856, 990)
Avg_Sales (856, 990)


In [60]:
first_key = list(feature_dict.keys())[0]
#test_data = np.expand_dims(feature_dict[list(feature_dict.keys())[0]].values,2)
for i in feature_dict.keys():
    if i == first_key:
        test_data = np.expand_dims(feature_dict[i].values[:,-160:],2)
        val_data = np.expand_dims(feature_dict[i].values[:,-208:-48],2)
        train_data = np.expand_dims(feature_dict[i].values[:,:-96],2)
    else:
        tmp_test = np.expand_dims(feature_dict[i].values[:,-160:],2)
        tmp_val = np.expand_dims(feature_dict[i].values[:,-208:-48],2)
        tmp_train = np.expand_dims(feature_dict[i].values[:,:-96],2)
        test_data = np.concatenate([test_data,tmp_test],2)
        val_data = np.concatenate([val_data,tmp_val],2)
        train_data = np.concatenate([train_data,tmp_train],2)

In [61]:
test_data.shape,train_data.shape,val_data.shape

((856, 160, 16), (856, 894, 16), (856, 160, 16))

In [66]:
feature_info = {i:None for i in feature_dict.keys()}
for k in feature_info.keys():
    size = len(full_data[k].unique())
    feature_info[k] = (feature_index[k],size, int(np.ceil(size/2)) if size < 20 else 10)

In [64]:
feature_index

{'Assortment': 14,
 'Avg_Sales': 15,
 'DayOfWeek': 0,
 'Id': 1,
 'Open': 2,
 'Promo': 3,
 'Sales': 4,
 'SchoolHoliday': 5,
 'StateHoliday': 6,
 'Store': 7,
 'StoreType': 13,
 'dom': 8,
 'doy': 9,
 'month': 10,
 'quarter': 11,
 'week': 12}

In [67]:
feature_info

{'Assortment': (14, 3, 2),
 'Avg_Sales': (15, 856, 10),
 'DayOfWeek': (0, 7, 4),
 'Id': (1, 41090, 10),
 'Open': (2, 2, 1),
 'Promo': (3, 2, 1),
 'Sales': (4, 20028, 10),
 'SchoolHoliday': (5, 2, 1),
 'StateHoliday': (6, 4, 2),
 'Store': (7, 856, 10),
 'StoreType': (13, 4, 2),
 'dom': (8, 31, 10),
 'doy': (9, 365, 10),
 'month': (10, 12, 6),
 'quarter': (11, 4, 2),
 'week': (12, 52, 10)}

In [68]:
with open('../data/cubic_data.pickle','wb') as f:
    pickle.dump((feature_info,train_data,val_data,test_data),f)

In [69]:
train_data[:,:,feature_info['Sales'][0]]

array([[ 0.        ,  8.61812391,  8.37286082, ...,  8.19643681,
         8.21500643,  8.35631997],
       [ 0.        ,  8.82820109,  8.68321598, ...,  8.640649  ,
         8.72939712,  8.45276133],
       [ 0.        ,  9.01736223,  8.8862709 , ...,  8.86883549,
         9.04676206,  8.79346036],
       ...,
       [ 0.        ,  8.7353644 ,  8.62407255, ...,  8.69148258,
         8.60355436,  8.53836743],
       [ 0.        ,  9.93513156,  9.82357817, ...,  9.87411023,
         9.83670652, 10.02087051],
       [ 0.        ,  8.21554741,  8.36590508, ...,  8.5596778 ,
         8.57847642,  8.95376929]])

In [43]:
feature_dict['Sales'].iloc[:,-160:].values.shape

(856, 160)

In [49]:
train_dict = {i:None for i in feature_dict.keys()}
val_dict = {i:None for i in feature_dict.keys()}
test_dict = {i:None for i in feature_dict.keys()}
size_dict = {i:None for i in feature_dict.keys()}

In [50]:
train_bunch = []
for k in test_dict.keys():
    test_dict[k] = feature_dict[k].iloc[:,-160:]
    val_dict[k] = feature_dict[k].iloc[:,-208:-48]
for i in range(-1*feature_dict['Sales'].shape[1],-208,50):
    tmp_dict = train_dict.copy()
    for k in test_dict.keys():
        tmp_dict[k] = feature_dict[k].iloc[:,i:i+160]
    train_bunch.append(tmp_dict)

In [51]:
for k in size_dict.keys():
    size = len(full_data[k].unique())
    size_dict[k] = (size, int(np.ceil(size/2)) if size < 20 else 10)

In [52]:
with open('../data/data_done.pickle','wb') as f:
    pickle.dump((train_bunch,val_dict,test_dict,size_dict),f)

In [53]:
with open('../data/data_done.pickle','rb') as f:
    a,b,c,d = pickle.load(f)

In [54]:
d

{'Assortment': (3, 2),
 'DayOfWeek': (7, 4),
 'Id': (41090, 10),
 'Open': (2, 1),
 'Promo': (2, 1),
 'Sales': (20028, 10),
 'SchoolHoliday': (2, 1),
 'StateHoliday': (4, 2),
 'Store': (856, 10),
 'StoreType': (4, 2),
 'dom': (31, 10),
 'doy': (365, 10),
 'month': (12, 6),
 'quarter': (4, 2),
 'week': (52, 10)}

In [55]:
b['Sales']

Sales                                                         \
Date   2015-02-22 2015-02-23 2015-02-24 2015-02-25 2015-02-26 2015-02-27   
Store_                                                                     
0         0.00000   8.188411   8.307706   8.262559   8.304742   8.457231   
1         0.00000   8.585786   8.513787   8.444838   8.696009   8.637817   
2         0.00000   8.861634   8.840001   8.713582   8.964184   9.097172   
3         0.00000   8.512783   8.588397   8.607765   8.756368   8.723557   
4         0.00000   8.811950   8.698181   8.698181   8.744966   8.859789   
5         0.00000   8.584665   8.485290   8.377471   8.474912   8.623893   
6         0.00000   8.732305   8.670944   8.864040   8.877661   8.980676   
7         0.00000   8.681520   8.517793   8.679992   8.687611   8.625330   
8         0.00000   8.078688   8.009363   8.087025   8.296547   8.686936   
9         0.00000   8.554682   8.313607   8.447629   8.502283   8.623713   
10        0.00000   8.521584   8.489411   8.432942   8.745125   8.841737   
11        0.00000   8.756682   8.861350   8.843759   8.908289   8.869820   
12        0.00000   8.656433   8.737934   8.709300   8.752423   9.071653   
13        0.00000   8.961494   8.871225   8.929435   9.072571   8.890548   
14        0.00000   8.216628   8.248267   8.073715   8.107419   8.385032   
15        0.00000   8.224967   8.170186   8.127405   8.216899   8.472196   
16        0.00000   7.977968   8.278682   8.241967   8.454679   8.735364   
17        0.00000   8.824825   8.900276   8.866017   8.931023   9.268326   
18        0.00000   9.179159   9.125218   9.245708   9.194211   9.484177   
19        0.00000   8.748622   8.815222   8.882114   8.926252   9.102755   
20        0.00000   8.764678   8.641356   8.698014   8.705497   8.979165   
21        0.00000   8.053251   8.165932   8.110127   8.451908   8.333991   
22        0.00000   8.327968   8.273337   8.421343   8.561401   8.679822   
23        0.00000   8.187855   8.173293   8.010692   8.058327   8.129470   
24        0.00000   8.867005   8.736971   8.757784   8.963288   9.138844   
25        0.00000   8.907342   8.742734   8.870523   8.984568   9.167433   
26        0.00000   9.154193   9.072916   8.909235   9.041803   9.145055   
27        0.00000   8.213653   8.237215   8.293800   8.413165   8.627302   
28        0.00000   8.190077   8.216358   8.238801   8.183118   8.543640   
29        0.00000   8.107720   8.125335   8.066521   8.262301   8.106816   
...           ...        ...        ...        ...        ...        ...   
826       0.00000   8.140024   8.260493   8.344980   8.317522   8.611594   
827       0.00000   8.561975   8.632662   8.584104   8.530109   8.664233   
828       0.00000   8.622814   8.668540   8.711608   8.654517   8.868273   
829       0.00000   9.098179   9.059052   9.109967   9.181220   9.135293   
830       0.00000   8.063063   8.382518   8.432071   8.364042   8.563313   
831       0.00000   8.399085   8.417815   8.476788   8.674710   8.802973   
832       0.00000   8.139732   8.283999   8.387540   8.419360   8.356320   
833       0.00000   8.463792   8.444838   8.494743   8.685923   8.951699   
834       0.00000   8.235361   8.465057   8.366138   8.425955   8.442901   
835       0.00000   9.004300   8.970051   8.941938   9.076237   9.315691   
836       0.00000   8.878219   8.742095   8.779096   8.815073   9.012133   
837       0.00000   9.282754   9.264450   9.276315   9.172015   9.305923   
838       0.00000   8.258422   8.263075   8.157370   8.188967   8.362642   
839       0.00000   8.488382   8.355615   8.410498   8.565602   9.006509   
840       9.44034   9.165238   9.217217   9.139489   9.117567   9.289614   
841       0.00000   8.779711   8.738575   8.743532   8.745284   8.764366   
842       0.00000   8.125039   8.266935   8.234830   8.252967   8.555452   
843       0.00000   8.971956   8.938532   8.926916   9.044758   9.137232   
844       0.00000   8.695674   8.669399   8.658693   8.77693

In [56]:
len(a)

16

In [217]:
test[test.Store==115].shape

(48, 14)

In [218]:
def get_lookback(data,n=65,f=5):
    length = len(data)
    result = []
    for i in range(n,length,f):
        result.append(data[i-n:i])
    return np.expand_dims(np.array(result),2)

In [220]:
full_data.Id.min()

-50000

In [191]:
len(set(train.Store)),len(set(test.Store))

(856, 856)

In [226]:
get_lookback(train.Sales[train.Store==1],n=160,f=48).shape

(17, 160, 1)

In [221]:
full_data.columns

Index(['Date', 'DayOfWeek', 'Id', 'Open', 'Promo', 'Sales', 'SchoolHoliday',
       'StateHoliday', 'Store', 'dom', 'doy', 'month', 'quarter', 'week',
       'StoreType', 'Assortment'],
      dtype='object')

In [285]:
attr_list = ['Id','Sales','DayOfWeek',  'Open', 'Promo',  'SchoolHoliday',
       'StateHoliday', 'Store', 'dom', 'doy', 'month', 'quarter', 'week',
       'StoreType', 'Assortment']
store_list = full_data.Store.unique()

In [289]:
store_list.shape

(856,)

In [231]:
np.concatenate?

In [292]:
def print_data(data):
    '''
    def get_one_store(col,s):
        return get_lookback(data[col][data.Store==s],160,48)
    
    result = get_one_store(attr_list[0],store_list[0])
    print(result.shape)
    print(get_one_store(attr_list[0],store_list[1]).shape)
    '''
    result = get_lookback(data[attr_list[0]][data.Store==store_list[0]],n=160,f=48)
    for s in store_list[1:]:
        print(get_lookback(data[attr_list[0]][data.Store==s],n=160,f=48).shape)
        result = np.concatenate((result,get_lookback(data[attr_list[0]][data.Store==s],n=160,f=48)),axis=0)
    for col in attr_list[1:]:
        store_tmp = get_lookback(data[col][data.Store==store_list[0]],n=160,f=48)
        for s in store_list[1:]:
            tmp = get_lookback(data[col][data.Store==s],n=160,f=48)
            #print(tmp.shape,store_tmp.shape)
            store_tmp = np.concatenate((store_tmp,tmp),axis=0)
        result = np.concatenate((result,store_tmp),axis=2)
    
    return result

In [296]:
data = print_data(full_data)

(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(14, 160, 1)
(14, 160, 1)
(18, 160, 1)
(18, 160, 1)
(14, 160, 1)
(18, 160, 1)
(14, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(14, 160, 1)
(14, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(14, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(14, 160, 1)
(18, 160, 1)
(14, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(14, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(14, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(14, 160, 1)
(18, 160, 1)
(18, 160, 1)
(14, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(14, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(14, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)

(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(14, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(14, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(14, 160, 1)
(18, 160, 1)
(18, 160, 1)
(14, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(14, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(14, 160, 1)
(14, 160, 1)
(14, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(14, 160, 1)
(18, 160, 1)
(14, 160, 1)
(18, 160, 1)
(14, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(14, 160, 1)
(14, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(14, 160, 1)
(14, 160, 1)
(18, 160, 1)
(18, 160, 1)
(18, 160, 1)
(14, 160, 1)
(18, 160, 1)
(18, 160, 1)
(14, 160, 1)
(18, 160, 1)
(18, 160, 1)

In [300]:
np.transpose(data,[2,0,1]).shape

(15, 14688, 160)

In [298]:
data[:,:,[data[:,:,0]==50000]].shape

IndexError: too many indices for array

In [294]:
result = get_lookback(full_data.StoreType[full_data.Store==store_list[0]],n=160,f=48)
result2 = get_lookback(full_data.StoreType[full_data.Store==store_list[1]],n=160,f=48)

In [295]:
for s in store_list[1:]:
    print(result.shape,s)
    result = np.concatenate((result,get_lookback(full_data.StoreType[full_data.Store==s],n=160,f=48)),axis=0)

(18, 160, 1) 854
(36, 160, 1) 853
(54, 160, 1) 852
(72, 160, 1) 851
(90, 160, 1) 850
(104, 160, 1) 849
(118, 160, 1) 848
(136, 160, 1) 847
(154, 160, 1) 846
(168, 160, 1) 845
(186, 160, 1) 844
(200, 160, 1) 843
(218, 160, 1) 842
(236, 160, 1) 841
(254, 160, 1) 840
(272, 160, 1) 839
(290, 160, 1) 838
(304, 160, 1) 837
(318, 160, 1) 836
(336, 160, 1) 835
(354, 160, 1) 834
(372, 160, 1) 833
(390, 160, 1) 832
(408, 160, 1) 831
(426, 160, 1) 830
(444, 160, 1) 829
(458, 160, 1) 828
(476, 160, 1) 827
(494, 160, 1) 826
(512, 160, 1) 825
(530, 160, 1) 824
(548, 160, 1) 823
(566, 160, 1) 822
(584, 160, 1) 821
(602, 160, 1) 820
(620, 160, 1) 819
(634, 160, 1) 818
(652, 160, 1) 817
(666, 160, 1) 816
(684, 160, 1) 815
(702, 160, 1) 814
(720, 160, 1) 813
(738, 160, 1) 812
(756, 160, 1) 811
(774, 160, 1) 810
(792, 160, 1) 809
(810, 160, 1) 808
(824, 160, 1) 807
(842, 160, 1) 806
(860, 160, 1) 805
(878, 160, 1) 804
(896, 160, 1) 803
(914, 160, 1) 802
(928, 160, 1) 801
(946, 160, 1) 800
(964, 160, 1) 7

(7924, 160, 1) 392
(7938, 160, 1) 391
(7956, 160, 1) 390
(7974, 160, 1) 389
(7992, 160, 1) 388
(8010, 160, 1) 387
(8028, 160, 1) 386
(8046, 160, 1) 385
(8064, 160, 1) 384
(8082, 160, 1) 383
(8100, 160, 1) 382
(8114, 160, 1) 381
(8128, 160, 1) 380
(8146, 160, 1) 379
(8160, 160, 1) 378
(8178, 160, 1) 377
(8196, 160, 1) 376
(8214, 160, 1) 375
(8228, 160, 1) 374
(8246, 160, 1) 373
(8260, 160, 1) 372
(8278, 160, 1) 371
(8296, 160, 1) 370
(8314, 160, 1) 369
(8328, 160, 1) 368
(8346, 160, 1) 367
(8364, 160, 1) 366
(8378, 160, 1) 365
(8396, 160, 1) 364
(8414, 160, 1) 363
(8432, 160, 1) 362
(8446, 160, 1) 361
(8464, 160, 1) 360
(8482, 160, 1) 359
(8500, 160, 1) 358
(8518, 160, 1) 357
(8536, 160, 1) 356
(8554, 160, 1) 355
(8572, 160, 1) 354
(8590, 160, 1) 353
(8608, 160, 1) 352
(8622, 160, 1) 351
(8640, 160, 1) 350
(8658, 160, 1) 349
(8676, 160, 1) 348
(8694, 160, 1) 347
(8712, 160, 1) 346
(8730, 160, 1) 345
(8748, 160, 1) 344
(8766, 160, 1) 343
(8784, 160, 1) 342
(8802, 160, 1) 341
(8820, 160, 

In [284]:
store_list[-1]

1115